# HW 3
## Ermanov Danat 154

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm
import itertools
from node2vec import Node2Vec

In [2]:
Data = pd.read_csv("data3.csv")
Data['Source'].unique()

array(['Theoretical and Applied Genetics',
       'The International Journal of Advanced Manufacturing Technology',
       'Osteoporosis International', 'Oecologia',
       'World Journal of Surgery', 'Scientometrics',
       'Quality of Life Research', 'Planta',
       'Journal of Autism and Developmental Disorders',
       'Metallurgical and Materials Transactions B', 'Euphytica',
       'Cellulose', 'Journal of Gastrointestinal Surgery',
       'Climatic Change', 'BMC Cancer', 'Pediatric Nephrology'],
      dtype=object)

In [3]:
data = Data[(Data['Source'] == 'Theoretical and Applied Genetics') | (Data['Source'] == 'Osteoporosis International')].copy()
#data = Data[Data['Source'] == 'World Journal of Surgery'].copy()

In [4]:
data

,Authors,Citated IDs,ID,Issue,Keywords,Source,Title,Volume,Year,N of Pages
0,"Capell, T.; Escobar, C.; Liu, H.; Burtin, D.; ...",NaN,10.1007/s001220050892,1-2,"Key words Transgenic plants, Development, Oryz...",Theoretical and Applied Genetics,Over-expression of the oat arginine decarboxyl...,97.0,1998,9
1,"Tosti, Nicola; Negri, Valeria",NaN,10.1007/s00122-005-1964-1,7,"Amplify Fragment Length Polymorphism, Minimum ...",Theoretical and Applied Genetics,On-going on-farm microevolutionary processes i...,110.0,2005,9
2,"Mazzucato, Andrea; Papa, Roberto; Bitocchi, El...","10.1007/s001220100662, 10.1007/s00122-001-0775...",10.1007/s00122-007-0699-6,5,"Simple Sequence Repeat Marker, Fruit Weight, F...",Theoretical and Applied Genetics,"Genetic diversity, structure and marker-trait ...",116.0,2008,13
3,"Yahiaoui, S.; Igartua, Ernesto; Moralejo, M.; ...","10.1023/A:1009602321815, 10.1023/A:10086625150...",10.1007/s00122-007-0665-3,2,"Climatic Factor, Electronic Supplementary Mate...",Theoretical and Applied Genetics,Patterns of genetic and eco-geographical diver...,116.0,2008,12
4,"Gauthier, P.; Gouesnard, B.; Dallard, J.; Reda...",NaN,10.1007/s00122-002-0903-7,1,Zea mays L. Bulk RFLP European landraces Genet...,Theoretical and Applied Genetics,RFLP diversity and relationships among traditi...,105.0,2002,9
5,"Fu, Yong Bi; Peterson, Gregory W.; Williams, D...",NaN,10.1007/s00122-005-2044-2,3,"Amplify Fragment Length Polymorphism, Amplify ...",Theoretical and Applied Genetics,Patterns of AFLP variation in a core subset of...,111.0,2005,10
6,"Dobrovolskaya, O.; Saleh, U.; Malysheva-Otto, ...","10.1023/B:GRES.0000024164.80444.f0, 10.1007/s0...",10.1007/s00122-005-0061-9,7,"Duplicates, Ex situ genebank collections, Micr...",Theoretical and Applied Genetics,Rationalising germplasm collections: a case st...,111.0,2005,8
7,"Reif, Jochen C.; Hamrit, Sonia; Heckenberger, ...","10.1007/s001220050776, 10.1023/A:1007571101815...",10.1007/s00122-005-0016-1,5,"Linkage Disequilibrium, Simple Sequence Repeat...",Theoretical and Applied Genetics,Genetic structure and diversity of European fl...,111.0,2005,8
8,"Russell, J. R.; Booth, A.; Fuller, J. D.; Baum...","10.1007/s001220050445, 10.1007/s001220051486, ...",10.1007/s00122-003-1261-9,3,"Barley, Landraces, Microsatellites, Linkage di...",Theoretical and Applied Genetics,Patterns of polymorphism detected in the chlor...,107.0,2003,9
9,"Laperche, A.; Devienne-Barret, F.; Maury, O.; ...","10.1023/A:1009669507144, 10.1007/s00122-004-19...",10.1007/s00122-006-0373-4,6,"Double Haploid Line, Linkage Group, Total Root...",Theoretical and Applied Genetics,A simplified conceptual model of carbon/nitrog...,113.0,2006,16


In [5]:
def graph_assemble(d, year1, year2): # code from group project
    data = d.copy()
    data["Authors"] = data["Authors"].apply(lambda x: x.split("; "))
    data["Authors"] = data["Authors"].apply(lambda x: x[:200] if len(x) > 200 else x)

    d1 = data["Authors"].loc[(data.Year >= year1) & (data.Year < year2)].values
    d2 = data["Authors"].loc[data.Year == year2].values

    coauthors1, coauthors2 = {}, {}

    for art in tqdm(d1):
        for x in itertools.permutations(art, r=2):
            coauthors1[x] = coauthors1.get(x, 0) + 1

    for art in tqdm(d2):
        for x in itertools.permutations(art, r=2):
            coauthors2[x] = coauthors2.get(x, 0) + 1

    G1, G2 = nx.Graph(), nx.Graph()
    G1.add_edges_from([key + tuple([{'weight': value}]) for key,value in coauthors1.items()])
    G2.add_edges_from([key + tuple([{'weight': value}]) for key,value in coauthors2.items()])
    
    return G1, G2

In [ ]:
years = sorted(data['Year'].unique())
step = 10

df = pd.DataFrame(columns=['year',
                           'common_neighbors', 'jaccard_coefficient', 
                           'shortest_path_length', 
                           'density',
                           'target'])


for i in range(len(years) - step - 6):
    g1, g2 = graph_assemble(data, years[i], years[i+step])

    for v1, v2 in nx.complement(g1).edges():
        if (v1 in g2.nodes() and v2 in g2.nodes()):
            if (nx.has_path(g1, source=v1, target=v2)):
                path = nx.shortest_path_length(g1, source=v1, target=v2, weight='weight')
            else:
                path = -1
            df = df.append({'year': years[i],
                            'common_neighbors': len(list(nx.common_neighbors(g1, v1, v2))), 
                            'jaccard_coefficient': list(nx.jaccard_coefficient(g1, [(v1, v2)]))[0][2], 
                            'shortest_path_length': path, 
                            'density': nx.density(g1), 
                            'target': int((v1, v2) in g2.edges())}, ignore_index=True)
    print(years[i+5], len(df))
 
   

100%|██████████| 12/12 [00:00<00:00, 26658.71it/s]


1980 0
1981 0
1982 3
1983 6


100%|██████████| 19/19 [00:00<00:00, 58296.84it/s]

1984 24
1985 55



100%|██████████| 40/40 [00:00<00:00, 42366.71it/s]

...

1992 4169



100%|██████████| 81/81 [00:00<00:00, 22984.82it/s]

1993 10346



100%|██████████| 106/106 [00:00<00:00, 70247.47it/s]

1994 16424


In [252]:
df_test = pd.DataFrame(columns=['year','common_neighbors', 'jaccard_coefficient', 'shortest_path_length', 'density','target'])
g1, g3 = graph_assemble(data, years[len(years) - step - 5], years[len(years) - 5])
g2, g4 = graph_assemble(data, years[len(years) - 5], years[len(years) - 1])

for v1, v2 in nx.complement(g1).edges():
    if (v1 in g2.nodes() and v2 in g2.nodes()):
        if (nx.has_path(g1, source=v1, target=v2)):
            path = nx.shortest_path_length(g1, source=v1, target=v2, weight='weight')
        else:
            path = -1
        df_test = df_test.append({'year': years[len(years) - step - 5],
                                  'common_neighbors': len(list(nx.common_neighbors(g1, v1, v2))), 
                                  'jaccard_coefficient': list(nx.jaccard_coefficient(g1, [(v1, v2)]))[0][2], 
                                  'shortest_path_length': path, 
                                  'density': nx.density(g1), 
                                  'target': int((v1, v2) in g2.edges())}, ignore_index=True)

100%|██████████| 19/19 [00:00<00:00, 7135.72it/s]


In [272]:
len(df[df['target'] == 1])

90

In [237]:
len(df)

12078

Have some problems with node2vec

In [227]:
g1, g2 = graph_assemble(data, years[len(years) - step - 5], years[len(years) - 5])

node2vec = Node2Vec(g1, dimensions=64, walk_length=30, num_walks=200, workers=4)  # Use temp_folder for big graphs

# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)


from node2vec.edges import HadamardEmbedder

edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

print(edges_embs[('Gupta, Manish K.', 'Minocha, Vivek Ratan')])

Computing transition probabilities: 100%|██████████| 2341/2341 [00:00<00:00, 2993.14it/s]


TypeError: __init__() got an unexpected keyword argument 'require'

## 3.

In [239]:
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model

In [256]:
yTrain = df.target
XTrain = df.drop(['target'], axis=1)
yTest = df_test.target
XTest = df_test.drop(['target'], axis=1)

In [264]:
def score(yTest, y):
    print('recall: ', recall_score(yTest, y))
    print('mean_squared_error: ', mean_squared_error(yTest, y))
    print('precision: ', precision_score(yTest, y))
    print('accuracy: ', accuracy_score(yTest, y))
    print('f1_score: ', f1_score(yTest, y))
    
    return 

In [268]:
model = SVC()
model.fit(XTrain, yTrain)
y = model.predict(XTest)
print('SVC')
score(yTest, y)

SVC
recall:  0.0
precision:  0.0
accuracy:  0.9658634538152611
f1_score:  0.0
mean_squared_error:  0.03413654618473896


/home/danat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/danat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [269]:
model = linear_model.LogisticRegression()
model.fit(XTrain, yTrain)
y = model.predict(XTest)
print('LogisticRegression')
score(yTest, y)

LogisticRegression
recall:  0.0
precision:  0.0
accuracy:  0.9658634538152611
f1_score:  0.0
mean_squared_error:  0.03413654618473896


/home/danat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/danat/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [270]:
model = DecisionTreeClassifier() #Decision Tree
model.fit(XTrain, yTrain)
y = model.predict(XTest)
print('DecisionTreeClassifier')
score(yTest, y)

DecisionTreeClassifier
recall:  0.0
precision:  0.0
accuracy:  0.9618473895582329
f1_score:  0.0
mean_squared_error:  0.03815261044176707
